In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.pipeline import Pipeline
warnings.filterwarnings("ignore")


In [84]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

In [3]:
df_train

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [4]:
df_test

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [6]:
df_train.isnull().sum(), df_test.isnull().sum() 

(id             0
 keyword       61
 location    2533
 text           0
 target         0
 dtype: int64,
 id             0
 keyword       26
 location    1105
 text           0
 dtype: int64)

In [7]:
df_train.drop(['location','keyword','id'],axis=1,inplace=True)
df_test.drop(['location','keyword','id'],axis=1,inplace=True)

In [8]:
df_train

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...
7608,Two giant cranes holding a bridge collapse int...,1
7609,@aria_ahrary @TheTawniest The out of control w...,1
7610,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,Police investigating after an e-bike collided ...,1


In [9]:
df_test

,text
0,Just happened a terrible car crash
1,"Heard about #earthquake is different cities, s..."
2,"there is a forest fire at spot pond, geese are..."
3,Apocalypse lighting. #Spokane #wildfires
4,Typhoon Soudelor kills 28 in China and Taiwan
...,...
3258,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,Storm in RI worse than last hurricane. My city...
3260,Green Line derailment in Chicago http://t.co/U...
3261,MEG issues Hazardous Weather Outlook (HWO) htt...


#### First level of text cleanup removing special characters and unwanted symbols and lowering every word using Regex.

In [10]:
def remove_symbols_low(dfcolumns):
    dfcolumns = dfcolumns.str.replace(r'\W',' ')
    dfcolumns = dfcolumns.str.lower()
    return dfcolumns

In [11]:
df_test.text = remove_symbols_low(df_test.text)
df_test

,text
0,just happened a terrible car crash
1,heard about earthquake is different cities s...
2,there is a forest fire at spot pond geese are...
3,apocalypse lighting spokane wildfires
4,typhoon soudelor kills 28 in china and taiwan
...,...
3258,earthquake safety los angeles ûò safety faste...
3259,storm in ri worse than last hurricane my city...
3260,green line derailment in chicago http t co u...
3261,meg issues hazardous weather outlook hwo htt...


In [12]:
df_train.text = remove_symbols_low(df_train.text)
df_train

,text,target
0,our deeds are the reason of this earthquake m...,1
1,forest fire near la ronge sask canada,1
2,all residents asked to shelter in place are ...,1
3,13 000 people receive wildfires evacuation or...,1
4,just got sent this photo from ruby alaska as ...,1
...,...,...
7608,two giant cranes holding a bridge collapse int...,1
7609,aria_ahrary thetawniest the out of control w...,1
7610,m1 94 01 04 utc 5km s of volcano hawaii htt...,1
7611,police investigating after an e bike collided ...,1


#### Second level of cleaning

###### Cleaning digits 

In [13]:
def num_remove(data):
    data = re.sub('[^a-zA-Z]',' ',data)
    data = data.split()
    return data
df_test.text = df_test.text.apply(num_remove)
df_test

,text
0,"[just, happened, a, terrible, car, crash]"
1,"[heard, about, earthquake, is, different, citi..."
2,"[there, is, a, forest, fire, at, spot, pond, g..."
3,"[apocalypse, lighting, spokane, wildfires]"
4,"[typhoon, soudelor, kills, in, china, and, tai..."
...,...
3258,"[earthquake, safety, los, angeles, safety, fas..."
3259,"[storm, in, ri, worse, than, last, hurricane, ..."
3260,"[green, line, derailment, in, chicago, http, t..."
3261,"[meg, issues, hazardous, weather, outlook, hwo..."


In [14]:
#apply it on train test aswell
df_train['text'] = df_train['text'].apply(num_remove)
df_train

,text,target
0,"[our, deeds, are, the, reason, of, this, earth...",1
1,"[forest, fire, near, la, ronge, sask, canada]",1
2,"[all, residents, asked, to, shelter, in, place...",1
3,"[people, receive, wildfires, evacuation, order...",1
4,"[just, got, sent, this, photo, from, ruby, ala...",1
...,...,...
7608,"[two, giant, cranes, holding, a, bridge, colla...",1
7609,"[aria, ahrary, thetawniest, the, out, of, cont...",1
7610,"[m, utc, km, s, of, volcano, hawaii, http, t, ...",1
7611,"[police, investigating, after, an, e, bike, co...",1


###### through the above method we have tokenize the sentences too.

### importing important specific natural language libraries

In [15]:
import nltk
import gensim
import normalise
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer ,CountVectorizer

In [16]:
#Calling instances of each APIs.
stem = PorterStemmer()
Wordnetlem = WordNetLemmatizer()
countVect = CountVectorizer()
tfidfVect = TfidfVectorizer()

In [17]:
# Removing stop words
sw = stopwords.words('english')
def remove_sw(data):
    data = [word for word in data if word not in sw]
    return data

In [18]:
df_test.text = df_test.text.apply(lambda x: remove_sw(x))
df_train.text = df_train.text.apply(lambda x: remove_sw(x))

In [19]:
df_test

,text
0,"[happened, terrible, car, crash]"
1,"[heard, earthquake, different, cities, stay, s..."
2,"[forest, fire, spot, pond, geese, fleeing, acr..."
3,"[apocalypse, lighting, spokane, wildfires]"
4,"[typhoon, soudelor, kills, china, taiwan]"
...,...
3258,"[earthquake, safety, los, angeles, safety, fas..."
3259,"[storm, ri, worse, last, hurricane, city, amp,..."
3260,"[green, line, derailment, chicago, http, co, u..."
3261,"[meg, issues, hazardous, weather, outlook, hwo..."


In [20]:
df_train

,text,target
0,"[deeds, reason, earthquake, may, allah, forgiv...",1
1,"[forest, fire, near, la, ronge, sask, canada]",1
2,"[residents, asked, shelter, place, notified, o...",1
3,"[people, receive, wildfires, evacuation, order...",1
4,"[got, sent, photo, ruby, alaska, smoke, wildfi...",1
...,...,...
7608,"[two, giant, cranes, holding, bridge, collapse...",1
7609,"[aria, ahrary, thetawniest, control, wild, fir...",1
7610,"[utc, km, volcano, hawaii, http, co, zdtoyd, ebj]",1
7611,"[police, investigating, e, bike, collided, car...",1


#### Now moving into some intermediate level of feature engineering part in NLP.

#### Lemmatization of words to convert it into its root words.

In [21]:
def lemmatize(data):
    data = [Wordnetlem.lemmatize(word) for word in data]
    return data

In [22]:
df_train["text"] = df_train["text"].apply(lambda x: lemmatize(x))

In [23]:
df_train

,text,target
0,"[deed, reason, earthquake, may, allah, forgive...",1
1,"[forest, fire, near, la, ronge, sask, canada]",1
2,"[resident, asked, shelter, place, notified, of...",1
3,"[people, receive, wildfire, evacuation, order,...",1
4,"[got, sent, photo, ruby, alaska, smoke, wildfi...",1
...,...,...
7608,"[two, giant, crane, holding, bridge, collapse,...",1
7609,"[aria, ahrary, thetawniest, control, wild, fir...",1
7610,"[utc, km, volcano, hawaii, http, co, zdtoyd, ebj]",1
7611,"[police, investigating, e, bike, collided, car...",1


In [24]:
df_test["text"] = df_test["text"].apply(lambda x: lemmatize(x))
df_test

,text
0,"[happened, terrible, car, crash]"
1,"[heard, earthquake, different, city, stay, saf..."
2,"[forest, fire, spot, pond, goose, fleeing, acr..."
3,"[apocalypse, lighting, spokane, wildfire]"
4,"[typhoon, soudelor, kill, china, taiwan]"
...,...
3258,"[earthquake, safety, los, angeles, safety, fas..."
3259,"[storm, ri, worse, last, hurricane, city, amp,..."
3260,"[green, line, derailment, chicago, http, co, u..."
3261,"[meg, issue, hazardous, weather, outlook, hwo,..."


#### Tokenizing words in the data set

### Creating a data processing check point

In [25]:
traindata = df_train.copy()
testdata = df_test.copy()

In [26]:
def joinwords(data):
    combined_text = ' '.join(data)
    return combined_text
combined = lambda x: joinwords(x)

In [27]:
traindata["text"] = traindata["text"].apply(combined)

In [28]:
testdata["text"] = testdata["text"].apply(combined)

In [29]:
traindata

,text,target
0,deed reason earthquake may allah forgive u,1
1,forest fire near la ronge sask canada,1
2,resident asked shelter place notified officer ...,1
3,people receive wildfire evacuation order calif...,1
4,got sent photo ruby alaska smoke wildfire pour...,1
...,...,...
7608,two giant crane holding bridge collapse nearby...,1
7609,aria ahrary thetawniest control wild fire cali...,1
7610,utc km volcano hawaii http co zdtoyd ebj,1
7611,police investigating e bike collided car littl...,1


In [38]:
testdata

,text
0,happened terrible car crash
1,heard earthquake different city stay safe ever...
2,forest fire spot pond goose fleeing across str...
3,apocalypse lighting spokane wildfire
4,typhoon soudelor kill china taiwan
...,...
3258,earthquake safety los angeles safety fastener ...
3259,storm ri worse last hurricane city amp others ...
3260,green line derailment chicago http co utbxlcbiuy
3261,meg issue hazardous weather outlook hwo http c...


### Converting text to Vectors /// Embedding.

##### seperating features and Target label.

In [30]:
X = traindata["text"]
X

0              deed reason earthquake may allah forgive u
1                   forest fire near la ronge sask canada
2       resident asked shelter place notified officer ...
3       people receive wildfire evacuation order calif...
4       got sent photo ruby alaska smoke wildfire pour...
                              ...                        
7608    two giant crane holding bridge collapse nearby...
7609    aria ahrary thetawniest control wild fire cali...
7610             utc km volcano hawaii http co zdtoyd ebj
7611    police investigating e bike collided car littl...
7612    latest home razed northern california wildfire...
Name: text, Length: 7613, dtype: object

In [40]:
y = df_train.target
y

0       1
1       1
2       1
3       1
4       1
       ..
7608    1
7609    1
7610    1
7611    1
7612    1
Name: target, Length: 7613, dtype: int64

##### Introducing testing set as X-test.

In [39]:
X_test = testdata.text
X_test

0                             happened terrible car crash
1       heard earthquake different city stay safe ever...
2       forest fire spot pond goose fleeing across str...
3                    apocalypse lighting spokane wildfire
4                      typhoon soudelor kill china taiwan
                              ...                        
3258    earthquake safety los angeles safety fastener ...
3259    storm ri worse last hurricane city amp others ...
3260     green line derailment chicago http co utbxlcbiuy
3261    meg issue hazardous weather outlook hwo http c...
3262    cityofcalgary activated municipal emergency pl...
Name: text, Length: 3263, dtype: object

#### importing Sklearn Model_selection API

In [34]:
from sklearn.model_selection import train_test_split ,cross_val_score

In [35]:
# We can intially use train_test_split method.
X_train, X_val, y_train, y_val = train_test_split(X,y ,test_size=0.3)

#### Model with Naive-Bayes MultiNomial NB its the most basic ML algo approach for this sort of task.


In [45]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

In [37]:
multNB = MultinomialNB()

#### Initially we can deploy a Pipeline and if further model improvemnt is needed we can move onto some specific stepwise precision approach.

In [50]:
model = Pipeline([
     ('vectorizer',TfidfVectorizer()),
     ('classifier',MultinomialNB())
])

In [51]:
model.fit(X_train,y_train)
score = model.score(X_val,y_val)
score

0.7985989492119089

In [52]:
pred = model.predict(X_val)

In [53]:
pred

array([0, 0, 1, ..., 1, 0, 0], dtype=int64)

In [80]:
from sklearn.metrics import confusion_matrix,accuracy_score,f1_score

In [58]:
confusion_matrix(pred,y_val)

array([[1184,  342],
       [ 118,  640]], dtype=int64)

In [83]:
f1_score(pred,y_val)

0.7356321839080461

### As the Accuracy is bit low we can move on to threshold adjustment or AUC_ROC correwction method

In [59]:
predproba = model.predict_proba(X_val)     #[:,1]
predproba

array([[0.55625726, 0.44374274],
       [0.63465246, 0.36534754],
       [0.21570927, 0.78429073],
       ...,
       [0.1968318 , 0.8031682 ],
       [0.70959959, 0.29040041],
       [0.52252806, 0.47747194]])

In [60]:
threshold = model.predict_proba(X_val)[:,1]
threshold

array([0.44374274, 0.36534754, 0.78429073, ..., 0.8031682 , 0.29040041,
       0.47747194])

###### reviewing accuracy

In [67]:
accuracy_ls = []
for thres in threshold:
    y_pred_review = np.where(threshold > thres ,1,0)
    score = accuracy_score(y_pred_review,y_val ,normalize=True)
    accuracy_ls.append(score)
thres_series = pd.Series(threshold)
accuracy_series = pd.Series(accuracy_ls)
fin_accuracy = pd.concat((thres_series,accuracy_series) ,axis=1)
fin_accuracy.columns = ['threshold','accuracy']
fin_accuracy.sort_values(by='accuracy',ascending=False,inplace=True)

In [68]:
fin_accuracy

,threshold,accuracy
827,0.523374,0.801664
2152,0.518687,0.801664
504,0.521124,0.801664
1652,0.513573,0.801664
501,0.523752,0.801664
...,...,...
1684,0.022491,0.432137
1665,0.019374,0.431699
272,0.019374,0.431699
1938,0.019352,0.430823


#### Final Adjusted prediction

In [72]:
final_adj_pred = np.where(threshold > 0.523374 ,1,0)
final_adj_pred

array([0, 0, 1, ..., 1, 0, 0])

In [73]:
confusion_matrix(final_adj_pred,y_val)

array([[1208,  359],
       [  94,  623]], dtype=int64)

In [81]:
f1_score(final_adj_pred,y_val)

0.733372572101236

In [74]:
scoreadj = accuracy_score(final_adj_pred,y_val)
scoreadj

0.8016637478108581

##### Eventhough its only a slight improvement in the score its satisfialbe for prediction.

In [79]:
final_test_data_pred = model.predict_proba(X_test)[:,1]
fin_adj_pred = (final_test_data_pred > 0.523374 ).astype(int)
fin_adj_pred

array([0, 0, 1, ..., 1, 1, 1])

### Creating a submission CSV file

In [86]:
dfsubmtest = pd.read_csv("test.csv")

In [98]:
submission =pd.DataFrame({'id':dfsubmtest.id,
                         'target':fin_adj_pred})
submission.set_index('id',inplace=True)

In [99]:
submission

,target
id,
0,0
2,0
3,1
9,1
11,1
...,...
10861,1
10865,0
10868,1


In [100]:
submission.to_csv("submission.csv")

## This is initial Submit more model improvement and feature engineering is on the way.

(5329, 16219)

  (0, 7814)	0.27008154532714523
  (0, 15014)	0.25858536622504386
  (0, 15372)	0.22499856320305722
  (0, 12492)	0.2532082692088281
  (0, 1198)	0.2327140477612539
  (0, 13460)	0.22696497401751772
  (0, 229)	0.3072774359722788
  (0, 5400)	0.2038689917149
  (0, 15150)	0.25196594951201184
  (0, 15114)	0.22373741762068478
  (0, 15390)	0.2571745660579662
  (0, 12213)	0.26823170215000197
  (0, 7950)	0.1735105100010922
  (0, 448)	0.31217893216255904
  (0, 6100)	0.35299576813694983
  (1, 7191)	0.34170244431898716
  (1, 8587)	0.3898962790636391
  (1, 5128)	0.3898962790636391
  (1, 7933)	0.22099831644910625
  (1, 3263)	0.2725869430745618
  (1, 9409)	0.3898962790636391
  (1, 9410)	0.3898962790636391
  (1, 7221)	0.3898962790636391
  (2, 4983)	0.4006095811907182
  (2, 2399)	0.07357620135735347
  :	:
  (5326, 1052)	0.37468698181942245
  (5327, 15087)	0.3598564310884222
  (5327, 13223)	0.3598564310884222
  (5327, 4676)	0.3598564310884222
  (5327, 12923)	0.3434399263308181
  (5327, 2433)	0.3434399263308